In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Note to visualize and compare the estimated lifetimes for DBA turbines from the RULe method's fatigue tables vs. structural reports

In [91]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from utils.setup_custom_logger import setup_custom_logger
import os 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [30 / 2.54, 20 / 2.54]
pd.set_option('display.width', 1000) # Print columns wider
pd.set_option('display.max_rows', 500) # Print more rows

Requirements: all other scripts has to be run including the storage parameters as True

In [92]:
logger = setup_custom_logger('lifetime_summary')    
res_base_dir = os.path.join(os.getcwd(), "output", "all_turbines")
df = pd.read_excel(os.path.join(res_base_dir, 'all_lifetimes_from_fatigue_tables.xlsx'), index_col = None) # remove _fromjson if necessary

In [93]:
print(df.groupby('cluster').count())
print(df.groupby('cluster')['lifetime'].mean())
print(df.groupby('cluster')['lifetime'].min())
print(df.groupby('cluster')['lifetime'].max())

         turbine_name  lifetime
cluster                        
JLN                26        26
JLO                47        47
JLP                22        22
cluster
JLN    34.400958
JLO    33.548858
JLP    47.470156
Name: lifetime, dtype: float64
cluster
JLN    34.400958
JLO    29.434783
JLP    44.466338
Name: lifetime, dtype: float64
cluster
JLN    34.400958
JLO    33.691951
JLP    56.752639
Name: lifetime, dtype: float64


In [94]:
fig1_w = go.FigureWidget()
fig1_w.layout = go.Layout(xaxis=dict(range=[int(df['lifetime'].min() * 0.8), int(df['lifetime'].max() * 1.1)]), title='Fatigue lifetime distribution of Dogger Bank A turbines according to RULe fatigue lookup tables', xaxis_title='Years', yaxis_title='Counts')
fig1_w.add_trace(go.Histogram(x = df[df['cluster'] == 'JLO']['lifetime'], opacity=0.8, name='Intermediate'))
fig1_w.add_trace(go.Histogram(x = df[df['cluster'] == 'JLN']['lifetime'], opacity=0.8, name='Deep'))
fig1_w.add_trace(go.Histogram(x = df[df['cluster'] == 'JLP']['lifetime'], opacity=0.8, name='Shallow'))
fig1_w.update_layout(barmode='stack')

FigureWidget({
    'data': [{'name': 'Intermediate',
              'opacity': 0.8,
              'type': 'histogram',
              'uid': 'ea51d10a-a94c-42d1-bc03-033519a61e17',
              'x': array([33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 31.9716647 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 32.94403893, 33.6919512 , 33.6919512 , 33.6919512 ,
  

In [95]:
df_report = pd.read_excel( os.path.join(res_base_dir, 'structural_report_inplace_lifetimes.xlsx'), index_col=None) # reported worst in place util

In [96]:
df_report_Dd_tot = pd.read_excel( os.path.join(res_base_dir, 'structural_report_Dd_tot_lifetimes.xlsx'), index_col=None) # reported worst Dd tot util

In [97]:
new_df = pd.DataFrame()

for name in df['turbine_name']:
    reported_elevation = df_report[df_report['turbine_name'] == name].copy()
    reported_elevation['lookup_table_lifetime'] = float(df[df['turbine_name'] == name]['lifetime'])
    new_df = pd.concat( [new_df, reported_elevation] )
    
new_df.columns

Index(['turbine_name', 'cluster', 'elevation', 'in_out', 'description', 'in_place_utilization', 'lifetime', 'lookup_table_lifetime'], dtype='object')

In [98]:
def identify_subseabed_points(descr):
    if 'CW' in descr:
        n = int(descr.split('CW-')[1])
        if n >= 10:
            return f'{descr} (Sub Seabed)'    
    return descr       

In [99]:
def plot_lifetime_histogram_based_on_point(lifetime_df, lifetime_id = 'lifetime', plot_title = 'design utilization'):

    fw = go.FigureWidget()
    fw.layout = go.Layout(xaxis=dict(range=[25,60]), title=f'Fatigue lifetime distribution of Dogger Bank A turbines according to {plot_title}', xaxis_title='Years', yaxis_title='No. of turbines')

    for descr in lifetime_df['description'].unique():
        these_df = lifetime_df[lifetime_df['description'] == descr]
        for cluster in these_df['cluster'].unique():
            fw.add_trace(go.Histogram(x = these_df[these_df['cluster'] == cluster][lifetime_id], opacity=0.8, name= f"{cluster} {identify_subseabed_points(descr)}"))
        
    fw.update_layout(barmode='stack')
    return fw

plot_lifetime_histogram_based_on_point(new_df, lifetime_id='lifetime')
    

FigureWidget({
    'data': [{'name': 'JLO Top Boat landing Support',
              'opacity': 0.8,
              'type': 'histogram',
              'uid': 'c7bdcb56-a19d-4eec-95d5-5427626ccae3',
              'x': array([33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546]

In [100]:
plot_lifetime_histogram_based_on_point(new_df, lifetime_id='lookup_table_lifetime', plot_title='RULe lookup tables')

FigureWidget({
    'data': [{'name': 'JLO Top Boat landing Support',
              'opacity': 0.8,
              'type': 'histogram',
              'uid': 'ebc3d9dd-ec31-4801-b95c-04e73c03a690',
              'x': array([33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512])},
             {'name': 'JLN Top Boat landing Support',
            

In [101]:
deviation_df = new_df[(new_df['lookup_table_lifetime'] - new_df['lifetime']) > 2.0 ]
deviation_df

,turbine_name,cluster,elevation,in_out,description,in_place_utilization,lifetime,lookup_table_lifetime


## Below there was made an attempt to correct the fatigue table of P57_DE based on it being confusiongly wrong

In [102]:
if False:
    turbine_to_correct = 'DA_P57_DE'
    increase_damage_by_factor = deviation_df['lookup_table_lifetime'] / deviation_df['lifetime']
    increase_damage_by_factor = np.float64(increase_damage_by_factor)    
    
    file_format = '.json' # xlsx or json
    file_name_key = 'lookup_table' # lookup_table or fatigue_damage
    file_loc = "all_turbines" # blob or all_turbines

    res_base_dir = os.path.join( os.getcwd(), "output", file_loc)
    paths_to_lookup_tables = [os.path.join(path, name) for path, subdirs, files in os.walk(res_base_dir) 
                                for name in files if ((file_name_key in name) and (file_format) in name)]

    path_to_lookup_table_lst = [p for p in paths_to_lookup_tables if turbine_to_correct in p]
    path_to_lookup_table = path_to_lookup_table_lst[0]
    
    lookup_table_df = pd.read_json(path_to_lookup_table, dtype=False)
    lookup_table_df.loc[:, lookup_table_df.columns.str.contains('sector')] = lookup_table_df.loc[:, lookup_table_df.columns.str.contains('sector')].astype(np.float64) * increase_damage_by_factor
    
    from lifetime_calculation_from_lookup import calculate_lifetime_from_fatigue_lookup_table
    new_lifetimes = calculate_lifetime_from_fatigue_lookup_table(lookup_table_df)
    print(new_lifetimes.min())
    
    lookup_table_df.to_excel(path_to_lookup_table.replace('.json', '.xlsx'), index = False)
    lookup_table_df.to_json(path_to_lookup_table, double_precision = 15, force_ascii = True, indent = 4)
    print(f"Stored new lookup table for {turbine_to_correct}")